In [ ]:
#import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
#train = np.load('../nn/data/train_sample.npy')
train = np.load('../nn/data/train_scaled.npy')
#test_data  = np.load('../nn/data/test_scaled.npy')

In [ ]:
labels = train[:,0].reshape(-1,1)
train = train[:,1:]

In [ ]:
#############################################################
# convert data into train_true, train_false, and validation #
#############################################################
train_true  = train[labels[:,0] == 1]
train_false = train[labels[:,0] == 0]

valid_size_true = int(train_true.shape[0] * .1)
valid_size_false = 10000 - valid_size_true

idx_true = range(train_true.shape[0])
np.random.shuffle(idx_true)

valid_data = train_true[idx_true[0:valid_size_true], :]
valid_labels = np.ones([valid_size_true,1])

idx_false = range(train_false.shape[0])
np.random.shuffle(idx_false)

valid_data   = np.append(valid_data, train_false[idx_false[0:valid_size_false] , :], axis=0)
valid_labels = np.append(valid_labels, np.zeros([valid_size_false,1]), axis=0 )

train_true = train_true[valid_size_true:, :]
train_false = train_false[valid_size_false:, :]
train_labels_T = np.ones([train_true.shape[0], 1])
train_labels_F = np.zeros([train_false.shape[0], 1])

print "all data :\t",train.shape
print "all labels:\t",labels.shape
print "train true data :\t",train_true.shape
print "train true labels:\t",train_labels_T.shape
print "train false data :\t",train_false.shape
print "train false labels:\t",train_labels_F.shape
print "valid data :\t",valid_data.shape
print "valid labels:\t",valid_labels.shape

In [ ]:
##########################################
# convert data into train and validation #
##########################################
valid_size = 10000
idx = range(train.shape[0])
np.random.shuffle(idx)

valid_data = train[idx[0:valid_size], :]
valid_labels = labels[idx[0:valid_size], :]

train_data = train[idx[valid_size:], :]
train_labels = labels[idx[valid_size:], :]

print "all data :\t",train.shape
print "all labels:\t",labels.shape
print "train data :\t",train_data.shape
print "train labels:\t",train_labels.shape
print "valid data :\t",valid_data.shape
print "valid labels:\t",valid_labels.shape

In [ ]:
np.float32(-3.3)

In [ ]:

def eval_gini(y_true, y_prob):
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_loss(y_true, y_prob) :
    return np.float32( - eval_gini(np.asarray(y_true[:,0], dtype=np.float32), 
                                   np.asarray(y_prob[:,0], dtype=np.float32)))

In [ ]:
####################################################
# extract batch_data and batch_label at each epoch #
####################################################
def extract_batch(i, sym=False, true_perc=0):
    if sym :
        # uses i, true_perc, train_true, train_false and batch_size
        n_t = int(batch_size * true_perc)
        n_f = batch_size - n_t
       
        offset_T = (i * n_t) % (train_true.shape[0] - n_t)
        offset_F = (i * n_f) % (train_false.shape[0] - n_f)
        
        batch_data = train_true[ offset_T : (offset_T+n_t), : ]
        batch_data = np.append(batch_data, train_false[offset_F : (offset_F+n_f), :], axis=0)
        
        batch_labels = np.ones([n_t,1])
        batch_labels = np.append(batch_labels, np.zeros([n_f,1]), axis=0)
        
    else :
        # uses i, train_data, train_labels, and batch_size
        offset = (i * batch_size) % (train_data.shape[0] - batch_size)
        batch_data = train_data[ offset : (offset+batch_size), : ]
        batch_labels = train_labels[ offset : (offset+batch_size), : ]
    
    # shuffle data
    shuffle = range(batch_size)
    np.random.shuffle(shuffle)
    batch_data = batch_data[shuffle]
    batch_labels = batch_labels[shuffle]
    
    return batch_data, batch_labels.reshape(-1,)

In [ ]:
###########################
# Neural Network's Grapth #
###########################
batch_size = 64
n_features = 123
starter_learning_rate = .1
keep_prob = 1
l1_s = 7
l2_s = 7
l3_s = 5
l4_s = 5
l5_s = 5
l6_s = 2
epsilons = [1, 1e-1, 1e-5]
epsilon = 1e-3
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, n_features])
    tf_train_labels  = tf.placeholder(tf.int32, shape=[batch_size])
    tf_valid_dataset = tf.constant(valid_data, dtype=tf.float32)
    #tf_test_dataset  = tf.constant(test_data, dtype=tf.float32)
    
    W1 = tf.Variable(tf.truncated_normal( [n_features, l1_s], stddev=np.sqrt(2.0/n_features) ))
    W2 = tf.Variable(tf.truncated_normal( [l1_s, l2_s], stddev=np.sqrt(2./l1_s) ))
    W3 = tf.Variable(tf.truncated_normal( [l2_s, l3_s], stddev=np.sqrt(2./l2_s) ))
    W4 = tf.Variable(tf.truncated_normal( [l3_s, l4_s], stddev=np.sqrt(2./l3_s) ))
    W5 = tf.Variable(tf.truncated_normal( [l4_s, l5_s], stddev=np.sqrt(2./l4_s) ))
    W6 = tf.Variable(tf.truncated_normal( [l5_s, l6_s], stddev=np.sqrt(2./l5_s) ))
    
    b1 = tf.Variable(tf.zeros(l1_s))
    b2 = tf.Variable(tf.zeros(l2_s))
    b3 = tf.Variable(tf.zeros(l3_s))
    b4 = tf.Variable(tf.zeros(l4_s))
    b5 = tf.Variable(tf.zeros(l5_s))
    b6 = tf.Variable(tf.zeros(l6_s))
    #---------------------------------------------------------
    def batch_norm_wrapper(X,is_training, decay=0.999) :
        scale = tf.Variable(tf.ones(X.shape[-1]))
        offset  = tf.Variable(tf.zeros(X.shape[-1]))
        pop_mean = tf.Variable(tf.zeros(X.shape[-1]), trainable=False)
        pop_variance = tf.Variable(tf.ones(X.shape[-1]), trainable=False)
        
        if is_training :
            batch_mean, batch_variance = tf.nn.moments(X,[0])
            train_mean = tf.assign(pop_mean, pop_mean * decay + batch_mean * (1 - decay))
            train_variance = tf.assign(pop_variance, pop_variance * decay + batch_variance * (1 - decay))
            with tf.control_dependencies([train_mean, train_variance]) :
                return tf.nn.batch_normalization(X, batch_mean, batch_variance, offset, scale, epsilon)
        else :
            return tf.nn.batch_normalization(X, pop_mean, pop_variance, offset, scale, epsilon)  
    #--------------------------------------------------------------------------
    def forward_propagation(X, is_training=True) :
        l1_logits = tf.matmul(X, W1) + b1
        l1_BN     = batch_norm_wrapper(l1_logits, is_training)
        l1_relu   = tf.nn.relu(l1_BN)
        if is_training : l1_relu = tf.nn.dropout(l1_relu, keep_prob)

        l2_logits = tf.matmul(l1_relu, W2) + b2
        l2_BN     = batch_norm_wrapper(l2_logits, is_training)
        l2_relu   = tf.nn.relu(l2_BN)
        if is_training : l2_relu = tf.nn.dropout(l2_relu, keep_prob)

        l3_logits = tf.matmul(l2_relu, W3) + b3
        l3_BN     = batch_norm_wrapper(l3_logits, is_training)
        l3_relu   = tf.nn.relu(l3_BN)
        if is_training : l3_relu = tf.nn.dropout(l3_relu, keep_prob)
            
        l4_logits = tf.matmul(l3_relu, W4) + b4
        l4_BN     = batch_norm_wrapper(l4_logits, is_training)
        l4_relu   = tf.nn.relu(l4_BN)
        if is_training : l4_relu = tf.nn.dropout(l4_relu, keep_prob)
            
        l5_logits = tf.matmul(l4_relu, W5) + b5
        l5_BN     = batch_norm_wrapper(l5_logits, is_training)
        l5_relu   = tf.nn.relu(l5_BN)
        if is_training : l5_relu = tf.nn.dropout(l5_relu, keep_prob)    
            

        l6_logits = tf.matmul(l5_relu, W6) + b6
    
        return l6_logits
    #----------------------------------------------------------------------
    logits = forward_propagation(tf_train_dataset)
    #---------------------------------------------------------------------------------------
    # LOSS
    # Applying Sigmoid function to get loss. 
    loss = tf.reduce_mean(\
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))  
    #----------------------------------------------------------------------------
    # L2 regularzation
    #regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4) +\
    #    tf.nn.l2_loss(W5) + tf.nn.l2_loss(W6)
    #loss = tf.reduce_mean(loss + beta * regularizers)
    #-----------------------------------------------------------------
    # Optimizer
    # Adam optimizer
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    #----------------------------------------------------------------------------
    # Predictions
    train_predictions = tf.nn.sigmoid(logits)
    
    #validation predictions
    logits = forward_propagation(tf_valid_dataset, is_training=False)
    valid_predictions = tf.nn.sigmoid(logits)
    
    #test data predictions
    #logits = forward_propagation(tf_test_dataset, is_training=False)
    #test_predictions = tf.nn.sigmoid(logits)


In [ ]:
#####################################
#   Neural Network's Session        #
#####################################
num_steps = int(5e5+1)
losses = []
#test_pred = np.zeros(test_data.shape[0])
st = 0;

with tf.Session(graph=graph) as sess :
    
    tf.global_variables_initializer().run()
    print "initialized"
    
    for i in range(num_steps) :
        
        batch_data, batch_labels = extract_batch(i, True, 0.5)
        #batch_data, batch_labels = train_data, train_labels.reshape(-1,)
        feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels }        
        _, l, train_pred = sess.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        

        #if i >= 0 :
        if i % int(1e4) == 0 :
            
            losses.append(l)
            
            train_gini_score = eval_gini(batch_labels, train_pred[:,1])
            train_f1_score = f1_score(y_true=batch_labels, y_pred=[ p.argmax() for p in train_pred ])

            valid_pred = valid_predictions.eval()
            valid_gini_score = eval_gini(valid_labels, valid_pred[:,1])
            valid_f1_score = f1_score(y_true=valid_labels, y_pred=[ p.argmax() for p in valid_pred ])
            
            print "\nDone %d steps with loss : \t%.4f" % (i, l)
            print "Gini on training data : \t%.4f\t and f1 score of : %.4f" %\
                (train_gini_score, train_f1_score)
            print "Gini on validation data : \t%.4f\t and f1 score of : %.4f" %\
                (valid_gini_score, valid_f1_score)
            
            #if i > int(1e5) :
                #st += 1
                #test_pred += test_predictions.eval()[:,1]
    #test_pred /= st

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(1,len(losses)),losses[1:])
plt.title("Model Loss")
plt.xlabel('epoch x 1000')
plt.ylabel('loss')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
len(range(0,1000*len(losses),1000))

In [ ]:
test_pred.shape

In [ ]:
eval_gini(batch_labels[:,0], train_pred[:,0])

In [ ]:
train_pred[train_pred < .9] = 0

In [ ]:
train_pred[np.isnan(train_pred)] = 0

In [ ]:
valid_pred[np.isnan(valid_pred)] = 0
            valid_pred[valid_pred >= 0.5] = 1
            valid_pred[valid_pred < 0.5] = 0

In [ ]:
ids = np.load('./data/test_ids.npy')

In [ ]:
with open('./data/submission.csv', 'wb') as f :
	f.write('id,target')
	for i,p in zip(ids,test_pred) :
		f.write('\n%d,%.6f'%(i,p))


In [ ]:
np.min([ p.argmax() for p in valid_pred ])

In [ ]:
np.save('./data/test_ids.npy', ids)

In [ ]:
valid_pred[0].argmax()

In [ ]:
a = [p[1] for p in test_pred]

In [ ]:
a = train_pred[:,0]

In [ ]:
a

In [ ]:
g = tf.Graph()

In [ ]:
with g.as_default():
    lab = tf.constant(batch_labels)
    pr = tf.constant(train_pred)
    #pr = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)

    a = tf.py_func(eval_gini, (lab[:,0], pr[:,0]), tf.double)
    print a

In [ ]:
with tf.Session(graph=g):
    a.eval()